# Simple Python agent workflow

![image info](../../images/agent_demo1.png)

Here is an example of a simple Python agent workflow using a Python agent with the following steps:

1. User Query:

The process starts when the user submits a query or request to the Python agent.

2. Code and Test Case Generation:

The agent interprets the user's query and generates the corresponding Python code. Alongside the code, the agent creates a test case to verify the functionality of the generated code.

3. Execution and Validation:

The agent attempts to run the generated code to ensure it executes without errors.
The agent then runs the test case to confirm that the code produces the correct output.

4. Retry Mechanism:

If the code fails to run or the test case does not pass, the agent initiates a retry.
It regenerates the code and test case, addressing any issues identified during the previous attempt.

5. Result Output:

Once the code runs successfully and passes the test case, the agent delivers the result to the user.


## In summary:
user query

-> agent generate code and test case

-> check if the code is runnable and the test is passed

  -> if no, retry and regenerate

  -> if yes output result



## Install dependencies

First we will install the python SDK and set our API key!

In [1]:
!pip install mistralai==1.0.0

In [3]:
import os
from mistralai import Mistral
import re

api_key = os.environ["MISTRAL_API_KEY"]

client = Mistral(api_key=api_key)

## Agent

You can create an Agent in https://console.mistral.ai/build/agents/new, for this notebook we will use mistral-large-2407 as the model powering our agents!

Here is the instruction provided to the Python agent we create:

```
You are a Python coding assistant that only outputs Python code without any explanations or comments.

Return one Python function for the given query and one test case using assertion.

Return Python code with two sections:

## Python function

## Test case

  """
```

After we create the agent, we will retrieve the Agents ID from the UI where we created the agent:



In [4]:
agent_id= "ag:8e2706f0:20240807:python-agent:ffdc79f8"

## Simple Python agent workflow

Now we can write several functions `run_python_agent`, `extract_code`, `check_code`, and combine them in a `run_workflow` function.

In [5]:
def run_python_agent(query):
    """
    Sends a user query to a Python agent and returns the response.

    Args:
        query (str): The user query to be sent to the Python agent.

    Returns:
        str: The response content from the Python agent.
    """
    print("### Run Python agent")
    print(f"User query: {query}")
    try:
        response = client.agents.complete(
            agent_id= agent_id,
            messages = [
                {
                    "role": "user",
                    "content":  query
                },
            ]
        )
        result = response.choices[0].message.content
        return result
    except Exception as e:
        print(f"Request failed: {e}. Please check your request.")
        return None

def extract_pattern(text, pattern):
    """
    Extracts a pattern from the given text.

    Args:
        text (str): The text to search within.
        pattern (str): The regex pattern to search for.

    Returns:
        str: The extracted pattern or None if not found.
    """
    match = re.search(pattern, text, flags=re.DOTALL)
    if match:
        return match.group(1).strip()
    return None

def extract_code(result):
    """
    Extracts Python function and test case from the response content.

    Args:
        result (str): The response content from the Python agent.

    Returns:
        tuple: A tuple containing the extracted Python function, test function, and a retry flag.
    """
    retry = False
    print("### Extracting Python code")
    python_function = extract_pattern(result, r'## Python function(.*?)## Test case')
    if not python_function:
        retry = True
        print("Python function failed to generate or wrong output format. Setting retry to True.")

    print("### Extracting test case")
    test_function = extract_pattern(result, r'## Test case(.*?)```')
    if not test_function:
        retry = True
        print("Test function failed to generate or wrong output format. Setting retry to True.")

    return python_function, test_function, retry

def check_code(python_function, test_function):
    """
    Executes the Python function and its test case, and checks for any errors.

    Args:
        python_function (str): The Python function to be executed.
        test_function (str): The test case to be executed.

    Returns:
        bool: A flag indicating whether the code execution needs to be retried.

    Warning:
        This code is designed to run code that’s been generated by a model, which may not be entirely reliable.
        It's strongly recommended to run this in a sandbox environment.
    """
    retry = False
    try:
        exec(python_function)
        print("Code executed successfully.")
        try:
            exec(test_function)
            print("Code passed test case.")
        except Exception:
            print(f"Test failed")
            retry = True
            print("Setting retry to True")
    except Exception:
        print(f"Code failed.")
        retry = True
        print("Setting retry to True")
    return retry

def run_workflow(query):
    """
    Runs the complete workflow to generate, extract, and validate Python code.

    Args:
        query (str): The user query to be processed.
    """
    print("### ENTER WORKFLOW")
    i = 0
    max_retries = 3
    retry = True # just to get it started
    while i < max_retries and retry:
        print(f"TRY # {i}")
        i += 1
        result = run_python_agent(query)
        python_function, test_function, retry = extract_code(result)
        retry = check_code(python_function, test_function)

    if not retry:
        print(f"Validated Python function: ```{python_function}```")
    print("### EXIT WORKFLOW")


In [7]:
run_workflow("How can I remove duplicates from a list")

### ENTER WORKFLOW
TRY # 0
### Run Python agent
User query: How can I remove duplicates from a list
### Extracting Python code
### Extracting test case
Code executed successfully.
Code passed test case.
Validated Python function: ```def remove_duplicates(lst):
    return list(set(lst))```
### EXIT WORKFLOW


In [8]:
run_workflow("How can I sort a list of words and add the word love to each of word ")

### ENTER WORKFLOW
TRY # 0
### Run Python agent
User query: How can I sort a list of words and add the word love to each of word 
### Extracting Python code
### Extracting test case
Code executed successfully.
Code passed test case.
Validated Python function: ```def sort_and_add_love(words):
    words.sort()
    return [word + 'love' for word in words]```
### EXIT WORKFLOW


In [ ]:
run_workflow("How can calculate fibinacci ")

### ENTER WORKFLOW
TRY # 0
### Extracting Python code
### Extracting test case
Code executed successfully.
Code passed test case.
Validated Python function: ```def fibonacci(n):
    if n <= 0:
        return []
    elif n == 1:
        return [0]
    elif n == 2:
        return [0, 1]

    fib_sequence = [0, 1]
    while len(fib_sequence) < n:
        fib_sequence.append(fib_sequence[-1] + fib_sequence[-2])
    return fib_sequence```
### EXIT WORKFLOW
